# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [1]:
# YOUR CHANGES HERE

%pip install -q pandas numpy

import pandas as pd
import numpy as np

df = pd.read_csv("strawberry-prices.tsv", sep="\t")

df.columns = [c.strip().lower() for c in df.columns]
if len(df.columns) != 2:
    raise ValueError(f"Expected 2 columns (month, price). Found: {df.columns.tolist()}")

df = df.rename(columns={df.columns[0]: "month", df.columns[1]: "price"})

df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)


train = df[(df["month"] >= "2020-01-01") & (df["month"] <= "2023-12-01")].copy()
test  = df[(df["month"] >= "2024-01-01") & (df["month"] <= "2024-12-01")].copy()

if len(test) != 12:
    raise ValueError(f"Expected 12 rows for 2024 (Jan–Dec). Got {len(test)} rows.")

train["moy"] = train["month"].dt.month
seasonal_means = train.groupby("moy")["price"].mean()

pred = test[["month"]].copy()
pred["moy"] = pred["month"].dt.month
pred["price"] = pred["moy"].map(seasonal_means)

actual = test["price"].to_numpy()
forecast = pred["price"].to_numpy()

mae = np.mean(np.abs(actual - forecast))
rmse = np.sqrt(np.mean((actual - forecast) ** 2))
mape = np.mean(np.abs((actual - forecast) / actual)) * 100

print("Backtest (2024) accuracy:")
print(f"  MAE :  {mae:.4f}")
print(f"  RMSE:  {rmse:.4f}")
print(f"  MAPE:  {mape:.2f}%")

out = pred[["month", "price"]].copy()

out.to_csv("strawberry-backtest.tsv", sep="\t", index=False)
print("Saved: strawberry-backtest.tsv")

out.head() 




[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Backtest (2024) accuracy:
  MAE :  0.1997
  RMSE:  0.2629
  MAPE:  5.21%
Saved: strawberry-backtest.tsv


,month,price
48,2024-01-01,4.36275
49,2024-02-01,4.09125
50,2024-03-01,3.68875
51,2024-04-01,3.78720
52,2024-05-01,3.52625


Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [2]:
# YOUR CHANGES HERE

tmp = test[["month", "price"]].merge(
    pred[["month", "price"]],
    on="month",
    suffixes=("_actual", "_pred")
).sort_values("month").reset_index(drop=True)

residuals = tmp["price_actual"].to_numpy() - tmp["price_pred"].to_numpy()

mean_resid = float(residuals.mean())

std_resid = float(residuals.std(ddof=1))

accuracy_df = pd.DataFrame({"mean": [mean_resid], "std": [std_resid]})
accuracy_df.to_csv("backtest-accuracy.tsv", sep="\t", index=False)

print("Saved: backtest-accuracy.tsv")
print("mean:", mean_resid)
print("std_ddof0:", float(residuals.std(ddof=0)))
print("std_ddof1:", float(residuals.std(ddof=1)))
accuracy_df


Saved: backtest-accuracy.tsv
mean: -0.004856250000000006
std_ddof0: 0.26287699874913767
std_ddof1: 0.27456607048151277


,mean,std
0,-0.004856,0.274566


Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [3]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

df = pd.read_csv("strawberry-prices.tsv", sep="\t")

df.columns = [c.strip().lower() for c in df.columns]
if len(df.columns) != 2:
    raise ValueError(f"Expected 2 columns (month, price). Found: {df.columns.tolist()}")

df = df.rename(columns={df.columns[0]: "month", df.columns[1]: "price"})
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)

train_all = df[(df["month"] >= "2020-01-01") & (df["month"] <= "2024-12-01")].copy()

train_all["moy"] = train_all["month"].dt.month
seasonal_means = train_all.groupby("moy")["price"].mean()

forecast_months = pd.date_range("2025-01-01", "2025-12-01", freq="MS")

forecast = pd.DataFrame({"month": forecast_months})
forecast["moy"] = forecast["month"].dt.month
forecast["price"] = forecast["moy"].map(seasonal_means)

out = forecast[["month", "price"]].copy()
out.to_csv("strawberry-forecast.tsv", sep="\t", index=False)

print("Saved: strawberry-forecast.tsv")
out.head() 


Saved: strawberry-forecast.tsv


,month,price
0,2025-01-01,4.50120
1,2025-02-01,4.12580
2,2025-03-01,3.69940
3,2025-04-01,3.74496
4,2025-05-01,3.46840


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [4]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

forecast = pd.read_csv("strawberry-forecast.tsv", sep="\t")
forecast["month"] = pd.to_datetime(forecast["month"])

BUDGET = 1_000_000
FREEZE_COST = 0.20
STORAGE_COST_PER_MONTH = 0.10
DISCOUNT = 0.90

results = []

forecast = forecast.sort_values("month").reset_index(drop=True)

for i, buy_row in forecast.iterrows():
    buy_month = buy_row["month"]
    buy_price = float(buy_row["price"])

    for j, sell_row in forecast.iterrows():
        sell_month = sell_row["month"]
        sell_price = float(sell_row["price"])

        if sell_month <= buy_month:
            continue

        months_held = (sell_month.year - buy_month.year) * 12 + (sell_month.month - buy_month.month)

        cost_per_pint = buy_price + FREEZE_COST + STORAGE_COST_PER_MONTH * months_held
        revenue_per_pint = sell_price * DISCOUNT
        profit_per_pint = revenue_per_pint - cost_per_pint

        pints_purchased = int(np.floor(BUDGET / cost_per_pint))
        expected_profit = pints_purchased * profit_per_pint

        results.append({
            "buy_month": buy_month.strftime("%Y-%m-01"),
            "sell_month": sell_month.strftime("%Y-%m-01"),
            "pints_purchased": pints_purchased,
            "expected_profit": float(expected_profit)
        })

timings = pd.DataFrame(results)

timings = timings.sort_values(["buy_month", "sell_month"]).reset_index(drop=True)

timings.to_csv("timings.tsv", sep="\t", index=False)
print("Saved: timings.tsv")
print("Rows in timings:", len(timings))
timings.head()


Saved: timings.tsv
Rows in timings: 66


,buy_month,sell_month,pints_purchased,expected_profit
0,2025-01-01,2025-02-01,208281,-226605.562380
1,2025-01-01,2025-03-01,204031,-320683.683940
2,2025-01-01,2025-04-01,199952,-326068.924672
3,2025-01-01,2025-05-01,196032,-388072.788480
4,2025-01-01,2025-06-01,192263,-443616.110420


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [5]:
# YOUR CHANGES HERE

timings = pd.read_csv("timings.tsv", sep="\t")
accuracy = pd.read_csv("backtest-accuracy.tsv", sep="\t")

best_row = timings.loc[timings["expected_profit"].idxmax()]
best_profit = best_row["expected_profit"]
one_std_profit = best_row["pints_purchased"] * float(accuracy.loc[0, "std"])

check = pd.DataFrame({"best_profit": [best_profit], "one_std_profit": [one_std_profit]})
check.to_csv("check.tsv", sep="\t", index=False)

print("Saved: check.tsv")
check


Saved: check.tsv


,best_profit,one_std_profit
0,114385.703292,70793.566481


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


In [6]:
ack_text = """I used ChatGPT (OpenAI) as a generative AI assistant to clarify assignment instructions, help debug Python code, and interpret results. All code was reviewed and executed by me.

No other outside sources or individuals were consulted.

Libraries used:
- pandas: data loading/manipulation and writing TSV outputs
- numpy: numeric calculations
"""

for fname in ["acknowledgments.txt", "acknowledgements.txt"]:
    with open(fname, "w") as f:
        f.write(ack_text)

print("Saved: acknowledgments.txt and acknowledgements.txt") 


Saved: acknowledgments.txt and acknowledgements.txt


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.